In [1]:
from jupyter_cadquery import set_defaults, open_viewer

set_defaults(theme="dark")
open_viewer("CadQuery")

import math
import cadquery as cq
from cq_gears import SpurGear, RingGear
import faulthandler

faulthandler.enable()
from jupyter_cadquery.viewer.client import show

Overwriting auto display for cadquery Workplane and Shape


CadViewerWidget(anchor='right', cad_width=800, glass=False, height=600, pinning=False, theme='dark', title='Ca…

In [84]:
main_arm_length = 125
secondary_arm_length = main_arm_length

arm_height = 10
arm_thickness = 10
arm_extra_length = 10

gear_size = main_arm_length
gear_height = 10

gear_hole = 10
gear_hole_tolerance = 0.5
gear_spoke_width = 15

In [85]:
main_arm = (
    cq.Workplane("XY")
    .center(main_arm_length / 2 - main_arm_length / 2 - arm_extra_length / 2, 0)
    .rect(main_arm_length + arm_extra_length * 2, arm_thickness * 2)
    .extrude(arm_height)
)

main_arm_gear_hub_1 = (
    main_arm.faces(">Z")
    .workplane()
    .center(-main_arm_length / 2, 0)
    .circle(gear_hole / 2 - gear_hole_tolerance)
    .extrude(gear_height * 2)
)

main_arm_gear_hub_2 = (
    main_arm.faces(">Z")
    .workplane()
    .center(main_arm_length / 2, 0)
    .circle(gear_hole / 2 - gear_hole_tolerance)
    .extrude(gear_height * 2)
)

arm = main_arm.union(main_arm_gear_hub_1).union(main_arm_gear_hub_2)
show(arm)

. sending ... done


In [86]:
gear_arm_forward = SpurGear(
    module=1.0,
    teeth_number=gear_size,
    width=gear_height,
    helix_angle=30,
    bore_d=gear_hole,
    n_spokes=5,
    spokes_id=35,
    spokes_od=gear_size - 30,
    spoke_width=gear_spoke_width,
    # spoke_fillet=4.0,
)
gear_arm_backward = SpurGear(
    module=1.0,
    teeth_number=gear_size,
    width=gear_height,
    helix_angle=-30,
    bore_d=gear_hole,
    n_spokes=5,
    spokes_id=35,
    spokes_od=gear_size - 30,
    spoke_width=gear_spoke_width,
    # spoke_fillet=4.0,
)
gear_arm_forward_2 = SpurGear(
    module=1.0,
    teeth_number=gear_size,
    width=gear_height,
    helix_angle=30,
    n_spokes=5,
    spokes_id=35,
    spokes_od=gear_size - 30,
    spoke_width=gear_spoke_width,
    # spoke_fillet=4.0,
)

#Top gear without magnet arm
arm_gear_1 = (
    arm.faces(">Z[1]")
    .workplane()
    .center(-gear_size / 2, 0)
    # .circle(gear_arm_forward.r0)
    # .extrude(-gear_arm_forward.width)
    .gear(gear_arm_forward)
)

#Top gear with magnet arm
arm_gear_2 = (
    arm.faces(">Z[1]")
    .workplane()
    .center(gear_size / 2, 0)
    # .circle(gear_arm_backward.r0)
    # .extrude(-gear_arm_backward.width)
    .gear(gear_arm_backward)
)
arm_gear_2_magnet_arm = (
    arm_gear_2.faces(">Z")
    .workplane()
    .center(main_arm_length / 2 + 5, 0)
    .rect(secondary_arm_length - 5, arm_thickness * 2)
    .extrude(arm_height)
)

#Bottom gear
arm_gear_3 = (
    arm.faces("<Z")
    .workplane()
    .center(-gear_size / 2, 0)
    # .circle(gear_motor_main.r0)
    # .extrude(gear_motor_main.width)
    .gear(gear_arm_forward_2)
)
arm_gear_3 = arm.union(arm_gear_3)

arm_gear_3_align_pin = (
    arm_gear_3.faces("<Z")
    .workplane()
    .center(-gear_size / 2, 0)
    .circle(gear_hole / 2 - gear_hole_tolerance)
    .extrude(20)
)

gears = arm_gear_1.union(arm_gear_2_magnet_arm).union(arm_gear_3_align_pin)
show(gears)

C:\Users\James\miniforge3\envs\cadquery\Lib\site-packages\cq_gears\spur_gear.py:348: RuntimeWarning: invalid value encountered in arcsin
  a2 = np.arcsin((spoke_width / 2.0) / (spokes_od / 2.0))


StdFail_NotDone: GC_MakeArcOfCircle::Value() - no result

In [ ]:
motor_gear_size = 20

gear_motor = SpurGear(
    module=1.0,
    teeth_number=motor_gear_size,
    width=gear_height,
    helix_angle=-30
)

main_motor = (
    cq.importers.importStep("nema17.step")
    .rotate((0, 0, 0), (0, 1, 0), 270)
    .translate([-main_arm_length / 2 - arm_extra_length / 2, -gear_size / 2 - motor_gear_size/2, -60])
)
secondary_motor = (
    cq.importers.importStep("nema17.step")
    .rotate((0, 0, 0), (0, 1, 0), 270)
    .translate([-main_arm_length / 2 - arm_extra_length / 2, gear_size / 2 + motor_gear_size/2, -30])
)

main_motor_gear = (
    arm.faces("<Z")
    .workplane()
    .gear(gear_motor)
)
main_motor_gear_rod = (
    main_motor_gear.faces("<Z")
    .workplane()    
    .circle(2.6)
    .circle(6)
    .extrude(30)
    .translate(
        [-main_arm_length / 2, -gear_size / 2 - motor_gear_size/2, 0]
    )
)

secondary_motor_gear = (
    arm.faces(">Z[1]")
    .workplane()
    .gear(gear_motor)
)
secondary_motor_gear_rod = (
    secondary_motor_gear.faces("<Z")
    .workplane()    
    .circle(2.6)
    .circle(6)
    .extrude(20)
    .translate(
        [-main_arm_length / 2, gear_size / 2 + motor_gear_size/2, 0]
    )
)

motors = (
    main_motor.union(secondary_motor).union(
    main_motor_gear_rod).union(secondary_motor_gear_rod)
)
show(motors)

In [ ]:
final = arm.union(gears).union(motors)

show(final, grid=[True, True, True])

In [ ]:
cq.exporters.export(arm_gear_1, "arm_gear_1.3mf", tolerance=0.01, angularTolerance=0.1)
cq.exporters.export(arm_gear_2_magnet_arm, "arm_gear_2.3mf", tolerance=0.01, angularTolerance=0.1)
cq.exporters.export(arm_gear_3_align_pin, "arm_gear_3.3mf", tolerance=0.01, angularTolerance=0.1)
cq.exporters.export(main_motor_gear_rod, "gear_motor_1.3mf", tolerance=0.01, angularTolerance=0.1)
cq.exporters.export(secondary_motor_gear_rod, "gear_motor_2.3mf", tolerance=0.01, angularTolerance=0.1)